In [153]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
import geopandas as gpd
import textwrap
from geojson import Feature, FeatureCollection, Point

In [154]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [155]:
df = pd.read_csv('Miami Neighborhoods New Construction - Wynwood.csv')

In [156]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('Wynwood.kml', driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\array.py:524: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return _unary_geo("centroid", self)
C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\geoseries.py:133: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


In [157]:
project_gdf

,Name,Description,geometry,center_point
0,Wynwood,,"POLYGON Z ((-80.19080 25.81133 0.00000, -80.19...",POINT (-80.19905 25.80439)


In [158]:
df

,address,developer,units,acerage,price_sale,price_loan,loan_financer,category,description,story_link,additional_sources
0,26 Northeast 27th Street,Ironstate Development and Brookfield Properties,289.0,1.3,15600000.0,NaN,NaN,proposed,"Hoboken, New Jersey-based Ironstate led by bro...",https://therealdeal.com/miami/2022/05/17/devel...,NaN
1,2825 Northwest Second Avenue,Gamma Real Estate,180.0,NaN,NaN,NaN,NaN,stalled,New York-based Gamma Real Estate took control ...,https://therealdeal.com/miami/2022/04/21/lende...,NaN
2,2000 North Miami Avenue,Clearline Real Estate,NaN,1.4,19100000.0,NaN,NaN,proposed,Clearline plans for a mixed-use project likely...,https://therealdeal.com/miami/2022/04/04/forme...,NaN
3,2021 Northwest Miami Court,Clearline Real Estate,NaN,1.4,19100000.0,NaN,NaN,proposed,Clearline plans for a mixed-use project likely...,https://therealdeal.com/miami/2022/04/04/forme...,NaN
4,37 Northeast 27th Street,Fifield Companies,210.0,1.4,19500000.0,NaN,NaN,proposed,"Fifield plans an eight-story, 210-unit apartme...",https://therealdeal.com/miami/2022/01/12/chica...,NaN
5,31 Northwest 29th Plaza,L&L Holding Company and Carpe Real Estate Part...,NaN,3.0,53000000.0,NaN,NaN,planned,Two New York developers plan a mixed-use proje...,https://therealdeal.com/miami/2022/01/03/devel...,NaN
6,94 Northeast 29th Street,Rilea Group and Promanas Group,127.0,0.6,12200000.0,NaN,NaN,planned,Rilea Group and Promanas Group plan to build 1...,https://therealdeal.com/miami/2021/10/15/rilea...,NaN
7,2700 Northwest Second Avenue,"TriStar Capital, Related Group, Lndmrk Develop...",300.0,1.3,26500000.0,20000000.0,Comerica Bank,planned,"TriStar Capital, Related Group, Lndmrk Develop...",https://therealdeal.com/miami/2021/09/23/david...,NaN
8,2431 Northwest Second Avenue,PMG and Greybook Realty Partners,318.0,NaN,57000000.0,142300000.0,Pacific Western Bank and Square Mile Capital,planned,PMG and Greybook Realty Partners scored a $142...,https://therealdeal.com/miami/2021/08/20/devel...,NaN
9,2130 North Miami Avenue,Related Group,317.0,NaN,1850000.0,NaN,NaN,????,Related Group plans to build a pair of 12-stor...,https://therealdeal.com/miami/2021/03/18/relat...,NaN


In [159]:
df['geo_address'] = df['address'] + ', Wynwood, Miami FL'

In [160]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [161]:
df['loc']

0     (26 NE 27th St, Miami, FL 33137, USA, (25.8023...
1     (2825 NW 2nd Ave, Miami, FL 33127, USA, (25.80...
2     (2000 N Miami Ave, Miami, FL 33127, USA, (25.7...
3     (2021 NW Miami Ct, Miami, FL 33127, USA, (25.7...
4     (37 NE 27th St, Miami, FL 33137, USA, (25.8028...
5     (31 NW 29th St, Miami, FL 33127, USA, (25.8040...
6     (94 NE 29th St, Miami, FL 33137, USA, (25.8037...
7     (2700 NW 2nd Ave, Miami, FL 33127, USA, (25.80...
8     (2431 NW 2nd Ave, Miami, FL 33127, USA, (25.80...
9     (2130 N Miami Ave, Miami, FL 33127, USA, (25.7...
10    (38 NW 22 St, Miami, FL 33127, USA, (25.797952...
11    (2805 NW 3rd Ave, Miami, FL 33127, USA, (25.80...
12    (129 NW 26th St, Miami, FL 33127, USA, (25.801...
Name: loc, dtype: object

In [162]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [163]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [164]:
df['image_img'] = ' '

In [173]:
df.at[0,'image_img']=('<img src="IronstateDevelopment.jpg" width="256" height="156">')
df.at[1,'image_img']=('<img src="GammaRealEstate.jpg" width="256" height="156">')
df.at[2,'image_img']=('<img src="Clearline.jpg" width="256" height="156">')
df.at[3,'image_img']=('<img src="Clearline.jpg" width="256" height="156">')
df.at[4,'image_img']=('<img src="fifield.jpg" width="256" height="156">')
df.at[5,'image_img']=('<img src="L&L_carpe.jpg" width="256" height="156">')
df.at[6,'image_img']=('<img src="Rilea_Promanas.jpg" width="256" height="156">')
df.at[7,'image_img']=('<img src="TriStar_related_and_co.jpg" width="256" height="156">')
df.at[8,'image_img']=('<img src="PMG_greybook.jpg" width="256" height="156">')
df.at[9,'image_img']=('<img src="related.jpg" width="256" height="156">')
df.at[10,'image_img']=('<img src="related.jpg" width="256" height="156">')
df.at[11,'image_img']=('<img src="relatedx2.jpg" width="256" height="156">')
df.at[12,'image_img']=('<img src="kushner.jpg" width="256" height="156">')

In [166]:
df.columns

Index(['address', 'developer', 'units', 'acerage', 'price_sale', 'price_loan',
       'loan_financer', 'category', 'description', 'story_link',
       'additional_sources', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude', 'image_img'],
      dtype='object')

How to customize HTML in folium popups:
https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a

In [175]:
def popup_html(row):
    i = row
    address = df['address'].iloc[i]
    developer = df['developer'].iloc[i]
    description = df['description'].iloc[i]
    image = df['image_img'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    {}'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


In [176]:
df

,address,developer,units,acerage,price_sale,price_loan,loan_financer,category,description,story_link,additional_sources,geo_address,loc,point,lat,lon,altitude,image_img
0,26 Northeast 27th Street,Ironstate Development and Brookfield Properties,289.0,1.3,15600000.0,NaN,NaN,proposed,"Hoboken, New Jersey-based Ironstate led by bro...",https://therealdeal.com/miami/2022/05/17/devel...,NaN,"26 Northeast 27th Street, Wynwood, Miami FL","(26 NE 27th St, Miami, FL 33137, USA, (25.8023...","(25.8023173, -80.19426910000001, 0.0)",25.802317,-80.194269,0.0,"<img src=""IronstateDevelopment.jpg"" width=""256..."
1,2825 Northwest Second Avenue,Gamma Real Estate,180.0,NaN,NaN,NaN,NaN,stalled,New York-based Gamma Real Estate took control ...,https://therealdeal.com/miami/2022/04/21/lende...,NaN,"2825 Northwest Second Avenue, Wynwood, Miami FL","(2825 NW 2nd Ave, Miami, FL 33127, USA, (25.80...","(25.8035697, -80.1989423, 0.0)",25.803570,-80.198942,0.0,"<img src=""GammaRealEstate.jpg"" width=""256"" hei..."
2,2000 North Miami Avenue,Clearline Real Estate,NaN,1.4,19100000.0,NaN,NaN,proposed,Clearline plans for a mixed-use project likely...,https://therealdeal.com/miami/2022/04/04/forme...,NaN,"2000 North Miami Avenue, Wynwood, Miami FL","(2000 N Miami Ave, Miami, FL 33127, USA, (25.7...","(25.795655, -80.1953415, 0.0)",25.795655,-80.195341,0.0,"<img src=""Clearline.jpg"" width=""256"" height=""1..."
3,2021 Northwest Miami Court,Clearline Real Estate,NaN,1.4,19100000.0,NaN,NaN,proposed,Clearline plans for a mixed-use project likely...,https://therealdeal.com/miami/2022/04/04/forme...,NaN,"2021 Northwest Miami Court, Wynwood, Miami FL","(2021 NW Miami Ct, Miami, FL 33127, USA, (25.7...","(25.7957879, -80.1955335, 0.0)",25.795788,-80.195533,0.0,"<img src=""Clearline.jpg"" width=""256"" height=""1..."
4,37 Northeast 27th Street,Fifield Companies,210.0,1.4,19500000.0,NaN,NaN,proposed,"Fifield plans an eight-story, 210-unit apartme...",https://therealdeal.com/miami/2022/01/12/chica...,NaN,"37 Northeast 27th Street, Wynwood, Miami FL","(37 NE 27th St, Miami, FL 33137, USA, (25.8028...","(25.8028775, -80.1942598, 0.0)",25.802878,-80.194260,0.0,"<img src=""fifield.jpg"" width=""256"" height=""156"">"
5,31 Northwest 29th Plaza,L&L Holding Company and Carpe Real Estate Part...,NaN,3.0,53000000.0,NaN,NaN,planned,Two New York developers plan a mixed-use proje...,https://therealdeal.com/miami/2022/01/03/devel...,NaN,"31 Northwest 29th Plaza, Wynwood, Miami FL","(31 NW 29th St, Miami, FL 33127, USA, (25.8040...","(25.8040987, -80.1958216, 0.0)",25.804099,-80.195822,0.0,"<img src=""L&L_carpe.jpg"" width=""256"" height=""1..."
6,94 Northeast 29th Street,Rilea Group and Promanas Group,127.0,0.6,12200000.0,NaN,NaN,planned,Rilea Group and Promanas Group plan to build 1...,https://therealdeal.com/miami/2021/10/15/rilea...,NaN,"94 Northeast 29th Street, Wynwood, Miami FL","(94 NE 29th St, Miami, FL 33137, USA, (25.8037...","(25.8037379, -80.1930806, 0.0)",25.803738,-80.193081,0.0,"<img src=""Rilea_Promanas.jpg"" width=""256"" heig..."
7,2700 Northwest Second Avenue,"TriStar Capital, Related Group, Lndmrk Develop...",300.0,1.3,26500000.0,20000000.0,Comerica Bank,planned,"TriStar Capital, Related Group, Lndmrk Develop...",https://therealdeal.com/miami/2021/09/23/david...,NaN,"2700 Northwest Second Avenue, Wynwood, Miami FL","(2700 NW 2nd Ave, Miami, FL 33127, USA, (25.80...","(25.8023458, -80.1996831, 0.0)",25.802346,-80.199683,0.0,"<img src=""TriStar_related_and_co.jpg"" width=""2..."
8,2431 Northwest Second Avenue,PMG and Greybook Realty Partners,318.0,NaN,57000000.0,142300000.0,Pacific Western Bank and Square Mile Capital,planned,PMG and Greybook Realty Partners scored a $142...,https://therealdeal.com/miami/2021/08/20/devel...,NaN,"2431 Northwest Second Avenue, Wynwood, Miami FL","(2431 NW 2nd Ave, Miami, FL 33127, USA, (25.80...","(25.8004148, -80.1988989, 0.0)",25.800415,-80.198899,0.0,"<img src=""PMG_greybook.jpg"" width=""256"" height..."
9,2130 North Miami Ave

In [170]:
# ,min_zoom=14.5, zoom_start=15, min_lat=25.83,max_lat=25.79,
#                scrollWheelZoom=True,
#               min_lon=-80.21,max_lon=-80.17, max_bounds=True

In [177]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Edgewater')

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [172]:
m.save('index.html')